## Correlation matrices and eigenvalues of merged per-core logs:

In [1]:
import os
import pandas as pd
import numpy as np
from numpy.linalg import eig, eigvalsh

import eigen_analysis

In [2]:
def get_latencies(out_fname):
    with open(out_fname, 'r') as f:
        lines = f.readlines()
    header = lines[0].rstrip('\n').split()
    read_lat = lines[1].rstrip('\n').split()
    lat = {'read': dict(zip(header[1:], [float(y) for y in read_lat[1:]]))}

    return lat['read']

In [3]:
def get_joules(df):
    joules_sum = df['joules_diff'].sum()
    joules = {'joules_sum': joules_sum}
    
    return joules

### Correlation Matrices: reflect dependencies/relationships between different columns of time-series data

In [23]:
corr_dvfs_counter_list = {}
eig_dvfs_counters_list = {}
corr_dvfs_non_counter_list = {}
eig_dvfs_non_counters_list = {}
joules_dvfs_list = {}

#for dvfs in ['0xd00', '0xf00', '0x1100', '0x1300', '0x1500', '0x1700', '0x1900', '0x1b00', '0x1d00']:
for dvfs in ['0x1100']:
    corr_qps_counter_list = {}
    corr_qps_non_counter_list = {}
    joules_qps_list = {}
    eig_qps_counters_list = {}
    eig_qps_non_counters_list = {}
    
#    for qps in ['200k', '400k', '600k']:
    for qps in ['600k']:
        merged_logs_dir = qps + '_merged/'
        
        corr_itr_counter_list = {}
        corr_itr_non_counter_list = {}
        joules_itr_list = {}
        eig_itr_counters_list = {}
        eig_itr_non_counters_list = {}
        
        all_itr_counter_dir = merged_logs_dir + qps + '_' + dvfs + '_counters_merged/archive/'
        all_itr_non_counter_dir = merged_logs_dir + qps + '_' + dvfs + '_non_counters_merged/archive/'

        print('Parsing counter metrics in ' + all_itr_counter_dir)
        for file in os.listdir(all_itr_counter_dir):
            
            !gunzip -v $all_itr_counter_dir$file
            
            if (os.path.isdir(all_itr_non_counter_dir + file) or (file == '.gitignore')):
                continue
            itr = file.split('_')[2]
            df = pd.read_csv(all_itr_counter_dir + file, sep = ',')
            joules = get_joules(df.copy())
            joules_itr_list[itr] = joules
            print('ITR_DELAY: ', itr, joules)
            df = df.drop([df.columns[0], 'joules_diff', 'timestamp'], axis=1)
            df_corr = df.corr()
            corr_itr_counter_list[itr] = df_corr
            
            vals_counters_dict = {}
            vals_counters, vecs_counter = eig(df_corr)
            i = 0
            for val in vals_counters:
                if val <= 0:
                    print('EIGENVAL <= 0')
                vals_counters_dict['eig_' + str(i)] = val
                i += 1
            eig_itr_counters_list[itr] = vals_counters_dict

            !gzip -v9 $all_itr_counter_dir$file
        
        print('Parsing non-counter metrics in ' + all_itr_non_counter_dir)
        for file in os.listdir(all_itr_non_counter_dir):
            
            !gunzip -v $all_itr_non_counter_dir$file
            
            if (os.path.isdir(all_itr_non_counter_dir + file) or (file == '.gitignore')):
                continue
            itr = file.split('_')[2]
            print('ITR_DELAY: ', itr)
            df = pd.read_csv(all_itr_non_counter_dir + file, sep= ',')
            df_corr = df.drop([df.columns[0], 'timestamp'], axis = 1).corr()
            corr_itr_non_counter_list[itr] = df_corr
            
            i = 4
            vals_non_counters_dict = {}
            vals_non_counters, vecs_non_counter = eig(df_corr)
            for val in vals_non_counters:
                if val <= 0:
                    print('EIGENVAL <= 0')
                vals_non_counters_dict['eig_' + str(i)] = val
                i += 1
            eig_itr_non_counters_list[itr] = vals_non_counters_dict

            !gzip -v9 $all_itr_non_counter_dir$file
            
        corr_qps_counter_list[qps] = corr_itr_counter_list
        corr_qps_non_counter_list[qps] = corr_itr_non_counter_list
        joules_qps_list[qps] = joules_itr_list
        eig_qps_counters_list[qps] = eig_itr_counters_list
        eig_qps_non_counters_list[qps] = eig_itr_non_counters_list
        
    corr_dvfs_counter_list[dvfs] = corr_qps_counter_list
    corr_dvfs_non_counter_list[dvfs] = corr_qps_non_counter_list
    joules_dvfs_list[dvfs] = joules_qps_list
    eig_dvfs_counters_list[dvfs] = eig_qps_counters_list
    eig_dvfs_non_counters_list[dvfs] = eig_qps_non_counters_list

Parsing counter metrics in 600k_merged/600k_0x1100_counters_merged/archive/
ITR_DELAY:  40 {'joules_sum': 51701.443894}
ITR_DELAY:  100 {'joules_sum': 50282.50123900005}
ITR_DELAY:  350 {'joules_sum': 46968.89126399995}
ITR_DELAY:  300 {'joules_sum': 47547.76753900007}
ITR_DELAY:  30 {'joules_sum': 52341.02242799991}
ITR_DELAY:  200 {'joules_sum': 48940.19977700003}
ITR_DELAY:  50 {'joules_sum': 51350.231513999985}
ITR_DELAY:  250 {'joules_sum': 48044.051461000054}
ITR_DELAY:  20 {'joules_sum': 52348.05920499994}
ITR_DELAY:  10 {'joules_sum': 52666.53941200003}
ITR_DELAY:  2 {'joules_sum': 52679.300368}
ITR_DELAY:  400 {'joules_sum': 47181.397756999955}
Parsing non-counter metrics in 600k_merged/600k_0x1100_non_counters_merged/archive/
ITR_DELAY:  250
ITR_DELAY:  350
ITR_DELAY:  30
ITR_DELAY:  20
ITR_DELAY:  400
ITR_DELAY:  100
ITR_DELAY:  300
ITR_DELAY:  2
ITR_DELAY:  50
ITR_DELAY:  200
ITR_DELAY:  10
ITR_DELAY:  40


In [32]:
# for dvfs in ['0x1100']:
#     corr_qps_counter_list = {}
#     corr_qps_non_counter_list = {}
#     joules_qps_list = {}
#     eig_qps_counters_list = {}
#     eig_qps_non_counters_list = {}
    
#     for qps in ['200k']:
#         merged_logs_dir = qps + '_merged/'
        
#         corr_itr_counter_list = {}
#         corr_itr_non_counter_list = {}
#         joules_itr_list = {}
#         eig_itr_counters_list = {}
#         eig_itr_non_counters_list = {}
        
#         all_itr_counter_dir = merged_logs_dir + qps + '_' + dvfs + '_counters_merged/archive/'
#         all_itr_non_counter_dir = merged_logs_dir + qps + '_' + dvfs + '_non_counters_merged/archive/'

#         print('Compressing counter metrics in ' + all_itr_counter_dir)
#         for file in os.listdir(all_itr_counter_dir):
#             !gzip -v9 $all_itr_counter_dir$file
#         print('Compressing non_counter metrics in ' + all_itr_non_counter_dir)
#         for file in os.listdir(all_itr_non_counter_dir):
#             !gzip -v9 $all_itr_non_counter_dir$file


In [24]:
# checking corr matrix
corr_dvfs_counter_list['0x1100']['600k']['300']

,instructions_diff,cycles_diff,ref_cycles_diff,llc_miss_diff
instructions_diff,1.000000,0.918864,0.921257,0.606254
cycles_diff,0.918864,1.000000,0.997620,0.777789
ref_cycles_diff,0.921257,0.997620,1.000000,0.771369
llc_miss_diff,0.606254,0.777789,0.771369,1.000000


In [26]:
# checking corr matrix
corr_dvfs_non_counter_list['0x1100']['600k']['100']

,rx_bytes,rx_desc,tx_bytes,tx_desc
rx_bytes,1.000000,0.688024,0.016659,0.024947
rx_desc,0.688024,1.000000,0.037897,0.047329
tx_bytes,0.016659,0.037897,1.000000,0.540762
tx_desc,0.024947,0.047329,0.540762,1.000000


In [27]:
# checking joules sum values
joules_dvfs_list['0x1100']['600k']

{'40': {'joules_sum': 51701.443894},
 '100': {'joules_sum': 50282.50123900005},
 '350': {'joules_sum': 46968.89126399995},
 '300': {'joules_sum': 47547.76753900007},
 '30': {'joules_sum': 52341.02242799991},
 '200': {'joules_sum': 48940.19977700003},
 '50': {'joules_sum': 51350.231513999985},
 '250': {'joules_sum': 48044.051461000054},
 '20': {'joules_sum': 52348.05920499994},
 '10': {'joules_sum': 52666.53941200003},
 '2': {'joules_sum': 52679.300368},
 '400': {'joules_sum': 47181.397756999955}}

In [28]:
eig_dvfs_counters_list['0x1100']['600k']

{'40': {'eig_0': 3.317612688915927,
  'eig_1': 0.4826246710068562,
  'eig_2': 0.19976180618786352,
  'eig_3': 8.338893551504741e-07},
 '100': {'eig_0': 3.4112449934277405,
  'eig_1': 0.4569844772045554,
  'eig_2': 0.13176856430803646,
  'eig_3': 1.9650596687614825e-06},
 '350': {'eig_0': 3.566010655662864,
  'eig_1': 0.3709717619970818,
  'eig_2': 0.06050016925881138,
  'eig_3': 0.0025174130812429638},
 '300': {'eig_0': 3.511624484626556,
  'eig_1': 0.41899301663287014,
  'eig_2': 0.0670564426499798,
  'eig_3': 0.002326056090592861},
 '30': {'eig_0': 3.9824994704001218,
  'eig_1': 0.013202361513610598,
  'eig_2': 0.004298087757765275,
  'eig_3': 8.032849971263602e-08},
 '200': {'eig_0': 3.4767208085024413,
  'eig_1': 0.42845679059735436,
  'eig_2': 0.09344702080250315,
  'eig_3': 0.0013753800977002332},
 '50': {'eig_0': 3.3491699641217973,
  'eig_1': 0.47724917044385523,
  'eig_2': 0.17358081229992733,
  'eig_3': 5.313441824299114e-08},
 '250': {'eig_0': 3.4906908477401313,
  'eig_1': 

In [29]:
eig_dvfs_non_counters_list['0x1100']['600k']

{'250': {'eig_4': 1.756652381428518,
  'eig_5': 1.5849532707932632,
  'eig_6': 0.2834096776246905,
  'eig_7': 0.3749846701535269},
 '350': {'eig_4': 1.782947665246136,
  'eig_5': 1.5807230863547777,
  'eig_6': 0.28493480541111404,
  'eig_7': 0.35139444298796807},
 '30': {'eig_4': 1.6691109451529667,
  'eig_5': 1.4387841788180953,
  'eig_6': 0.34885851323957506,
  'eig_7': 0.5432463627893642},
 '20': {'eig_4': 1.747176312965617,
  'eig_5': 1.3087857475989977,
  'eig_6': 0.3517141618134973,
  'eig_7': 0.59232377762189},
 '400': {'eig_4': 1.8000064912552936,
  'eig_5': 1.57526641228173,
  'eig_6': 0.28415550107284426,
  'eig_7': 0.34057159539013265},
 '100': {'eig_4': 1.7116653529408878,
  'eig_5': 1.5175727556892462,
  'eig_6': 0.31158545626323075,
  'eig_7': 0.45917643510663597},
 '300': {'eig_4': 1.7665839424729382,
  'eig_5': 1.5879219358651304,
  'eig_6': 0.28356640782381115,
  'eig_7': 0.36192771383812045},
 '2': {'eig_4': 1.9545816308844188,
  'eig_5': 1.1566428440500645,
  'eig_6'

### Latency Measurements:

In [34]:
lat_dvfs_list = {}
for dvfs in ['0xd00', '0xf00', '0x1100', '0x1300', '0x1500', '0x1700', '0x1900', '0x1b00', '0x1d00']:
    lat_qps_list = {}
    for qps in ['200k', '400k', '600k']:
        lat_itr_list = {}
        lat_dir = qps + '_qps/linux_mcd_out_0_' + dvfs + '_135_' + qps + '/'

        for itr in corr_dvfs_counter_list[dvfs][qps].keys():
            lat_file = lat_dir + 'linux.mcd.out.0_' + str(itr) + '_' + dvfs + '_135_' + qps[:-1] + '000'
            latencies = get_latencies(lat_file)
            lat_itr_list[itr] = latencies
        lat_qps_list[qps] = lat_itr_list
    lat_dvfs_list[dvfs] = lat_qps_list

In [35]:
lat_dvfs_list['0x1900']['200k']['100']

{'avg': 101.6,
 'std': 30.9,
 'min': 37.4,
 '5th': 53.2,
 '10th': 59.2,
 '50th': 101.9,
 '90th': 142.1,
 '95th': 151.6,
 '99th': 170.1}

### Putting it all together:

In [36]:
lat_columns = []
for itr in [list(lat_dvfs_list['0x1100']['200k'].keys())[0]]:
    lat_columns = list(lat_dvfs_list['0x1100']['200k'][itr].keys())
lat_columns

['avg', 'std', 'min', '5th', '10th', '50th', '90th', '95th', '99th']

In [37]:
eig_counters_columns = []
for itr in [list(eig_dvfs_counters_list['0x1100']['200k'].keys())[0]]:
    eig_counters_columns = list(eig_dvfs_counters_list['0x1100']['200k'][itr].keys())
eig_non_counters_columns = []
for itr in [list(eig_dvfs_non_counters_list['0x1100']['200k'].keys())[0]]:
    eig_non_counters_columns += list(eig_dvfs_non_counters_list['0x1100']['200k'][itr].keys())
eig_columns = eig_counters_columns + eig_non_counters_columns
eig_columns

['eig_0', 'eig_1', 'eig_2', 'eig_3', 'eig_4', 'eig_5', 'eig_6', 'eig_7']

In [38]:
all_cols = lat_columns + eig_columns + ['joules_sum']
all_cols.append('desc')
print(all_cols)

['avg', 'std', 'min', '5th', '10th', '50th', '90th', '95th', '99th', 'eig_0', 'eig_1', 'eig_2', 'eig_3', 'eig_4', 'eig_5', 'eig_6', 'eig_7', 'joules_sum', 'desc']


In [39]:
eig_dvfs_non_counters_list['0x1300']['200k']['10']

{'eig_4': 2.294147382104687,
 'eig_5': 0.9014327690853716,
 'eig_6': 0.5649305538527354,
 'eig_7': 0.23948929495720625}

In [42]:
eig_lat_joules_df = pd.DataFrame(columns=all_cols)
for dvfs in ['0xd00', '0xf00', '0x1100', '0x1300', '0x1500', '0x1700', '0x1900', '0x1b00', '0x1d00']:
    for qps in ['200k', '400k', '600k']:
        for itr in corr_dvfs_counter_list[dvfs][qps].keys():
            desc = '0' + '_' + itr + '_' + dvfs + '_' + '135' + '_' + qps
            new_row = {}
            for p in lat_columns:
                new_row[p] = lat_dvfs_list[dvfs][qps][itr][p]
            for e in eig_counters_columns:
                new_row[e] = eig_dvfs_counters_list[dvfs][qps][itr][e]
            for e in eig_non_counters_columns:
                new_row[e] = eig_dvfs_non_counters_list[dvfs][qps][itr][e]
            new_row['joules_sum'] = joules_dvfs_list[dvfs][qps][itr]['joules_sum']
            new_row['desc'] = desc
            eig_lat_joules_df = eig_lat_joules_df._append(new_row, ignore_index = True)

In [43]:
eig_lat_joules_df

,avg,std,min,5th,10th,50th,90th,95th,99th,eig_0,eig_1,eig_2,eig_3,eig_4,eig_5,eig_6,eig_7,joules_sum,desc
0,81.8,17.0,45.3,58.8,61.6,79.8,104.5,112.7,130.8,3.492777,0.199347,0.307877,1.926252e-08,1.874681,1.206367,0.388455,0.530496,35745.289533,0_40_0xd00_135_200k
1,179.3,57.8,45.3,86.6,100.6,180.9,256.3,270.5,297.3,3.608221,0.258202,0.133167,4.096448e-04,1.670789,1.540596,0.354743,0.433872,34341.416562,0_200_0xd00_135_200k
2,74.0,14.6,49.8,57.1,59.7,71.1,92.8,102.1,124.4,3.524457,0.190145,0.285398,2.713862e-08,2.161959,0.972311,0.308426,0.557303,35966.114230,0_20_0xd00_135_200k
3,292.5,113.5,49.8,111.7,135.7,294.9,442.9,471.3,515.1,3.593040,0.285410,0.121201,3.490791e-04,1.696883,1.574022,0.327540,0.401555,33724.280904,0_400_0xd00_135_200k
4,71.3,17.4,41.1,55.1,56.2,66.2,93.0,104.9,134.4,3.599618,0.245315,0.155067,8.947988e-08,2.069599,1.061065,0.347774,0.521561,36046.761964,0_2_0xd00_135_200k
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,57.5,17.2,34.0,42.0,43.3,53.5,75.6,86.5,123.3,3.956992,0.026499,0.016508,1.003158e-06,2.014633,1.106153,0.334646,0.544568,92513.073726,0_2_0x1d00_135_600k
320,206.0,71.4,41.1,92.7,110.4,205.8,300.6,322.0,358.2,3.423851,0.406464,0.125272,4.441266e-02,1.707851,1.596945,0.296460,0.398744,74429.830394,0_250_0x1d00_135_600k
321,71.1,14.7,34.0,49.2,52.7,71.0,88.0,94.9,110.9,3.879166,0.084843,0.035979,1.202505e-05,1.657982,1.498096,0.350948,0.492973,86921.377698,0_40_0x1d00_135_600k
322,106.0,29.2,37.4,60.0,66.7,106.3,145.2,152.8,168.8,3.513134,0.362429,0.120706,3.730351e-03,1.677405,1.543122,0.327947,0.451526,81377.862032,0_100_0x1d00_135_600k


In [47]:
outdir = 'eig_lat_joules_dfs/'
eig_lat_joules_df.to_csv(outdir + 'all_dvfs_qps_itr')

In [48]:
df = pd.read_csv(outdir + 'all_dvfs_qps_itr', index_col=0)

In [49]:
df

,avg,std,min,5th,10th,50th,90th,95th,99th,eig_0,eig_1,eig_2,eig_3,eig_4,eig_5,eig_6,eig_7,joules_sum,desc
0,81.8,17.0,45.3,58.8,61.6,79.8,104.5,112.7,130.8,3.492777,0.199347,0.307877,1.926252e-08,1.874681,1.206367,0.388455,0.530496,35745.289533,0_40_0xd00_135_200k
1,179.3,57.8,45.3,86.6,100.6,180.9,256.3,270.5,297.3,3.608221,0.258202,0.133167,4.096448e-04,1.670789,1.540596,0.354743,0.433872,34341.416562,0_200_0xd00_135_200k
2,74.0,14.6,49.8,57.1,59.7,71.1,92.8,102.1,124.4,3.524457,0.190145,0.285398,2.713862e-08,2.161959,0.972311,0.308426,0.557303,35966.114230,0_20_0xd00_135_200k
3,292.5,113.5,49.8,111.7,135.7,294.9,442.9,471.3,515.1,3.593040,0.285410,0.121201,3.490791e-04,1.696883,1.574022,0.327540,0.401555,33724.280904,0_400_0xd00_135_200k
4,71.3,17.4,41.1,55.1,56.2,66.2,93.0,104.9,134.4,3.599618,0.245315,0.155067,8.947988e-08,2.069599,1.061065,0.347774,0.521561,36046.761964,0_2_0xd00_135_200k
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,57.5,17.2,34.0,42.0,43.3,53.5,75.6,86.5,123.3,3.956992,0.026499,0.016508,1.003158e-06,2.014633,1.106153,0.334646,0.544568,92513.073726,0_2_0x1d00_135_600k
320,206.0,71.4,41.1,92.7,110.4,205.8,300.6,322.0,358.2,3.423851,0.406464,0.125272,4.441266e-02,1.707851,1.596945,0.296460,0.398744,74429.830394,0_250_0x1d00_135_600k
321,71.1,14.7,34.0,49.2,52.7,71.0,88.0,94.9,110.9,3.879166,0.084843,0.035979,1.202505e-05,1.657982,1.498096,0.350948,0.492973,86921.377698,0_40_0x1d00_135_600k
322,106.0,29.2,37.4,60.0,66.7,106.3,145.2,152.8,168.8,3.513134,0.362429,0.120706,3.730351e-03,1.677405,1.543122,0.327947,0.451526,81377.862032,0_100_0x1d00_135_600k
